In [2]:
import numpy as np
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import re
import math
import keras
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn import set_config
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import FunctionTransformer
from keras.wrappers.scikit_learn import KerasClassifier
# from scikeras.wrappers import KerasClassifier
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LayerNormalization
from keras.layers import Dropout
from keras.models import Sequential
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from pandas.api.types import is_numeric_dtype
from itertools import product
from joblib import dump
from scipy import stats
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer


In [3]:
df = pd.read_csv('/home/philippe/Documents/python project/EZStacking-main/dataset/iris.csv')

In [4]:
target_col = 'variety'

In [5]:
y = df[target_col]

In [6]:
X = df.drop(target_col, axis=1)

In [7]:
nb_features = len(X.columns.tolist())

In [8]:
nb_targets = len(y.unique())

In [9]:
layer_size = nb_features + nb_targets + 2

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [11]:
target_encoder = LabelEncoder() 
y_train = pd.Series(target_encoder.fit_transform(y_train)) 
y_test = pd.Series(target_encoder.transform(y_test))

In [12]:
num_ntree_processor = make_pipeline(SimpleImputer(strategy='mean', add_indicator=True), StandardScaler())

In [13]:
def K_Class(): 
    keras.backend.clear_session() 
#   neural network architecture: start 
    model = Sequential() 
    model.add(Dense(10 * layer_size, activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(Dropout(0.5)) 
#    model.add(LayerNormalization()) 
    model.add(Dense(layer_size, activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(Dropout(0.5)) 
#    model.add(LayerNormalization()) 
    model.add(Dense(nb_targets, activation='softmax')) 
#   neural network architecture: end   
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [14]:
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=20)

In [15]:
K_C = KerasClassifier(K_Class, batch_size=64, epochs=2000, callbacks=[es], validation_split=0.1, verbose=1) 
K_C._estimator_type = 'classifier'

In [16]:
level_0 = [ 
          ('LOGRL', make_pipeline(num_ntree_processor, LogisticRegression(solver='lbfgs', penalty='l2'))), 
          ('KERC', make_pipeline(num_ntree_processor, K_C)), 
          ]

In [18]:
level_1 = LogisticRegression(n_jobs=-1)

In [19]:
model = StackingClassifier(level_0, final_estimator=level_1)

In [20]:
%%time 
set_config(display='diagram') 
model.fit(X_train, y_train)

Epoch 1/2000
2/2 [==============================] - 2s 377ms/step - loss: 1.9132 - accuracy: 0.2500 - val_loss: 1.0621 - val_accuracy: 0.5000
Epoch 2/2000
2/2 [==============================] - 0s 70ms/step - loss: 1.7089 - accuracy: 0.3200 - val_loss: 1.0409 - val_accuracy: 0.5000
Epoch 3/2000
2/2 [==============================] - 0s 66ms/step - loss: 1.5934 - accuracy: 0.3200 - val_loss: 1.0214 - val_accuracy: 0.5833
Epoch 4/2000
2/2 [==============================] - 0s 86ms/step - loss: 1.4366 - accuracy: 0.3700 - val_loss: 1.0035 - val_accuracy: 0.5833
Epoch 5/2000
2/2 [==============================] - 0s 68ms/step - loss: 1.5316 - accuracy: 0.4400 - val_loss: 0.9887 - val_accuracy: 0.5833
Epoch 6/2000
2/2 [==============================] - 0s 66ms/step - loss: 1.4741 - accuracy: 0.4500 - val_loss: 0.9756 - val_accuracy: 0.6667
Epoch 7/2000
2/2 [==============================] - 0s 60ms/step - loss: 1.1309 - accuracy: 0.4600 - val_loss: 0.9635 - val_accuracy: 0.6667
Epoch 8/2000

Epoch 59/2000
2/2 [==============================] - 0s 40ms/step - loss: 0.3418 - accuracy: 0.9000 - val_loss: 0.6519 - val_accuracy: 0.9167
Epoch 60/2000
2/2 [==============================] - 0s 37ms/step - loss: 0.5403 - accuracy: 0.7600 - val_loss: 0.6474 - val_accuracy: 0.9167
Epoch 61/2000
2/2 [==============================] - 0s 43ms/step - loss: 0.3716 - accuracy: 0.8600 - val_loss: 0.6433 - val_accuracy: 0.9167
Epoch 62/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.3748 - accuracy: 0.8900 - val_loss: 0.6393 - val_accuracy: 0.9167
Epoch 63/2000
2/2 [==============================] - 0s 43ms/step - loss: 0.3659 - accuracy: 0.8800 - val_loss: 0.6352 - val_accuracy: 0.9167
Epoch 64/2000
2/2 [==============================] - 0s 53ms/step - loss: 0.3843 - accuracy: 0.8700 - val_loss: 0.6306 - val_accuracy: 0.9167
Epoch 65/2000
2/2 [==============================] - 0s 37ms/step - loss: 0.4122 - accuracy: 0.8400 - val_loss: 0.6259 - val_accuracy: 0.9167
Epoch 

2/2 [==============================] - 0s 57ms/step - loss: 0.2361 - accuracy: 0.9300 - val_loss: 0.4123 - val_accuracy: 0.9167
Epoch 117/2000
2/2 [==============================] - 0s 47ms/step - loss: 0.2211 - accuracy: 0.9400 - val_loss: 0.4105 - val_accuracy: 0.9167
Epoch 118/2000
2/2 [==============================] - 0s 58ms/step - loss: 0.2386 - accuracy: 0.9100 - val_loss: 0.4089 - val_accuracy: 0.9167
Epoch 119/2000
2/2 [==============================] - 0s 51ms/step - loss: 0.2067 - accuracy: 0.9600 - val_loss: 0.4068 - val_accuracy: 0.9167
Epoch 120/2000
2/2 [==============================] - 0s 42ms/step - loss: 0.2327 - accuracy: 0.9200 - val_loss: 0.4057 - val_accuracy: 0.9167
Epoch 121/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.2536 - accuracy: 0.9400 - val_loss: 0.4028 - val_accuracy: 0.9167
Epoch 122/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.2429 - accuracy: 0.9500 - val_loss: 0.4001 - val_accuracy: 0.9167
Epoch 123/2000

2/2 [==============================] - 0s 36ms/step - loss: 0.1841 - accuracy: 0.9600 - val_loss: 0.2993 - val_accuracy: 0.9167
Epoch 174/2000
2/2 [==============================] - 0s 39ms/step - loss: 0.2015 - accuracy: 0.9500 - val_loss: 0.2983 - val_accuracy: 0.9167
Epoch 175/2000
2/2 [==============================] - 0s 39ms/step - loss: 0.2492 - accuracy: 0.9200 - val_loss: 0.2967 - val_accuracy: 0.9167
Epoch 176/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.1753 - accuracy: 0.9600 - val_loss: 0.2954 - val_accuracy: 0.9167
Epoch 177/2000
2/2 [==============================] - 0s 37ms/step - loss: 0.2333 - accuracy: 0.9200 - val_loss: 0.2923 - val_accuracy: 0.9167
Epoch 178/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.1369 - accuracy: 0.9500 - val_loss: 0.2894 - val_accuracy: 0.9167
Epoch 179/2000
2/2 [==============================] - 0s 37ms/step - loss: 0.1764 - accuracy: 0.9500 - val_loss: 0.2862 - val_accuracy: 0.9167
Epoch 180/2000

2/2 [==============================] - 0s 35ms/step - loss: 1.0808 - accuracy: 0.5250 - val_loss: 0.9668 - val_accuracy: 0.6667
Epoch 11/2000
2/2 [==============================] - 0s 34ms/step - loss: 1.0017 - accuracy: 0.5625 - val_loss: 0.9552 - val_accuracy: 0.6667
Epoch 12/2000
2/2 [==============================] - 0s 32ms/step - loss: 1.2556 - accuracy: 0.4250 - val_loss: 0.9422 - val_accuracy: 0.6667
Epoch 13/2000
2/2 [==============================] - 0s 39ms/step - loss: 1.0843 - accuracy: 0.4375 - val_loss: 0.9293 - val_accuracy: 0.6667
Epoch 14/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.9071 - accuracy: 0.6250 - val_loss: 0.9185 - val_accuracy: 0.6667
Epoch 15/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.9035 - accuracy: 0.5875 - val_loss: 0.9079 - val_accuracy: 0.6667
Epoch 16/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.9521 - accuracy: 0.5500 - val_loss: 0.8970 - val_accuracy: 0.6667
Epoch 17/2000
2/2 [=

...

2/2 [==============================] - 0s 74ms/step - loss: 0.3027 - accuracy: 0.8765 - val_loss: 0.2701 - val_accuracy: 0.8889
Epoch 163/2000
2/2 [==============================] - 0s 59ms/step - loss: 0.2336 - accuracy: 0.8889 - val_loss: 0.2667 - val_accuracy: 0.8889
Epoch 164/2000
2/2 [==============================] - 0s 96ms/step - loss: 0.2339 - accuracy: 0.8765 - val_loss: 0.2633 - val_accuracy: 0.8889
Epoch 165/2000
2/2 [==============================] - 0s 71ms/step - loss: 0.2386 - accuracy: 0.9136 - val_loss: 0.2597 - val_accuracy: 0.8889
Epoch 166/2000
2/2 [==============================] - 0s 82ms/step - loss: 0.2074 - accuracy: 0.9012 - val_loss: 0.2573 - val_accuracy: 0.8889
Epoch 167/2000
2/2 [==============================] - 0s 90ms/step - loss: 0.1967 - accuracy: 0.9383 - val_loss: 0.2548 - val_accuracy: 0.8889
Epoch 168/2000
2/2 [==============================] - 0s 93ms/step - loss: 0.2691 - accuracy: 0.8889 - val_loss: 0.2520 - val_accuracy: 0.8889
Epoch 169/2000

2/2 [==============================] - 0s 66ms/step - loss: 0.1912 - accuracy: 0.9259 - val_loss: 0.1623 - val_accuracy: 1.0000
Epoch 220/2000
2/2 [==============================] - 0s 58ms/step - loss: 0.1936 - accuracy: 0.9753 - val_loss: 0.1597 - val_accuracy: 1.0000
Epoch 221/2000
2/2 [==============================] - 0s 52ms/step - loss: 0.1739 - accuracy: 0.9136 - val_loss: 0.1566 - val_accuracy: 1.0000
Epoch 222/2000
2/2 [==============================] - 0s 66ms/step - loss: 0.2046 - accuracy: 0.9259 - val_loss: 0.1534 - val_accuracy: 1.0000
Epoch 223/2000
2/2 [==============================] - 0s 71ms/step - loss: 0.2260 - accuracy: 0.9259 - val_loss: 0.1509 - val_accuracy: 1.0000
Epoch 224/2000
2/2 [==============================] - 0s 73ms/step - loss: 0.1498 - accuracy: 0.9753 - val_loss: 0.1488 - val_accuracy: 1.0000
Epoch 225/2000
2/2 [==============================] - 0s 85ms/step - loss: 0.1988 - accuracy: 0.9259 - val_loss: 0.1469 - val_accuracy: 1.0000
Epoch 226/2000

2/2 [==============================] - 0s 60ms/step - loss: 0.1557 - accuracy: 0.9259 - val_loss: 0.0968 - val_accuracy: 1.0000
Epoch 277/2000
2/2 [==============================] - 0s 52ms/step - loss: 0.1372 - accuracy: 0.9630 - val_loss: 0.0941 - val_accuracy: 1.0000
Epoch 278/2000
2/2 [==============================] - 0s 51ms/step - loss: 0.1334 - accuracy: 0.9506 - val_loss: 0.0914 - val_accuracy: 1.0000
Epoch 279/2000
2/2 [==============================] - 0s 85ms/step - loss: 0.1562 - accuracy: 0.9383 - val_loss: 0.0896 - val_accuracy: 1.0000
Epoch 280/2000
2/2 [==============================] - 0s 54ms/step - loss: 0.1869 - accuracy: 0.9383 - val_loss: 0.0878 - val_accuracy: 1.0000
Epoch 281/2000
2/2 [==============================] - 0s 74ms/step - loss: 0.2196 - accuracy: 0.9259 - val_loss: 0.0850 - val_accuracy: 1.0000
Epoch 282/2000
2/2 [==============================] - 0s 55ms/step - loss: 0.1863 - accuracy: 0.9136 - val_loss: 0.0817 - val_accuracy: 1.0000
Epoch 283/2000

2/2 [==============================] - 0s 69ms/step - loss: 0.1424 - accuracy: 0.9506 - val_loss: 0.0576 - val_accuracy: 1.0000
Epoch 334/2000
2/2 [==============================] - 0s 79ms/step - loss: 0.1039 - accuracy: 0.9630 - val_loss: 0.0577 - val_accuracy: 1.0000
Epoch 335/2000
2/2 [==============================] - 0s 62ms/step - loss: 0.1400 - accuracy: 0.9753 - val_loss: 0.0582 - val_accuracy: 1.0000
Epoch 336/2000
2/2 [==============================] - 0s 70ms/step - loss: 0.1696 - accuracy: 0.9136 - val_loss: 0.0580 - val_accuracy: 1.0000
Epoch 337/2000
2/2 [==============================] - 0s 64ms/step - loss: 0.1214 - accuracy: 0.9506 - val_loss: 0.0578 - val_accuracy: 1.0000
Epoch 338/2000
2/2 [==============================] - 0s 75ms/step - loss: 0.1315 - accuracy: 0.9753 - val_loss: 0.0575 - val_accuracy: 1.0000
Epoch 339/2000
2/2 [==============================] - 0s 59ms/step - loss: 0.1309 - accuracy: 0.9630 - val_loss: 0.0573 - val_accuracy: 1.0000
Epoch 340/2000

StackingClassifier(estimators=[('LOGRL',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('simpleimputer',
                                                                  SimpleImputer(add_indicator=True)),
                                                                 ('standardscaler',
                                                                  StandardScaler())])),
                                                ('logisticregression',
                                                 LogisticRegression())])),
                               ('KERC',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('simpleimputer',
                                                                  SimpleImputer(add_indicator=True)),
                                                                 ('standardscaler',
                                                                  StandardScaler())])),
                                                ('kerasclassifier',
                                                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fda1415fee0>)]))],
                   final_estimator=LogisticRegression(n_jobs=-1))

In [21]:
model.score(X_test, y_test)

2/2 [==============================] - 0s 7ms/step


0.868421052631579